In [1]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 600

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except: 
   pass

In [2]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'ir-engine-gcp'
!gcloud config set project {project_id}

data_bucket_name = 'bucket_ir_engine'
try:
    if os.environ["bucket_ir_engine"] is not None:
        pass  
except:
      !mkdir indices
      !mkdir bins
      !gsutil -u {project_id} cp gs://{data_bucket_name}/indices/* "indices/" 
      !gsutil -m rm -r gs://{data_bucket_name}/postings_gcp/Anchors_InvertedIndex/*.pickle
      !gsutil -m rm -r gs://{data_bucket_name}/postings_gcp/Title_InvertedIndex/*.pickle 
      !gsutil -m rm -r gs://{data_bucket_name}/postings_gcp/Body_InvertedIndex/*.pickle 
      !gsutil -u {project_id} cp -r gs://{data_bucket_name}/postings_gcp/* "bins/"


Updated property [core/project].
Copying gs://bucket_ir_engine/indices/Anchor_index.pkl...
Copying gs://bucket_ir_engine/indices/Body_index.pkl...
Copying gs://bucket_ir_engine/indices/Title_index.pkl...
\ [3 files][126.7 MiB/126.7 MiB]                                                
Operation completed over 3 objects/126.7 MiB.                                    
CommandException: 1 files/objects could not be removed.
Removing gs://bucket_ir_engine/postings_gcp/Title_InvertedIndex/101_posting_locs.pickle#1673080209138258...
Removing gs://bucket_ir_engine/postings_gcp/Title_InvertedIndex/107_posting_locs.pickle#1673080214126870...
Removing gs://bucket_ir_engine/postings_gcp/Title_InvertedIndex/100_posting_locs.pickle#1673080206031060...
Removing gs://bucket_ir_engine/postings_gcp/Title_InvertedIndex/103_posting_locs.pickle#1673080211490718...
Removing gs://bucket_ir_engine/postings_gcp/Title_InvertedIndex/102_posting_locs.pickle#1673080212052599...
Removing gs://bucket_ir_engine/postin

In [3]:
from storage_backend import *
import pickle
  
def posting_lists_iter(idx, dir):
  """ A generator that reads one posting list from disk and yields
      a (word:str, [(doc_id:int, tf:int), ...]) tuple.
  """
  with closing(MultiFileReader()) as reader:
      for w, locs in idx.posting_locs.items():
        locs = [(dir + loc[0], loc[1]) for loc in locs]
        print(locs)
        b = reader.read(locs, idx.df[w] * TUPLE_SIZE)
        posting_list = []
        for i in range(idx.df[w]):
            doc_id = int.from_bytes(b[i * TUPLE_SIZE:i * TUPLE_SIZE + 4], 'big')
            tf = int.from_bytes(b[i * TUPLE_SIZE + 4:(i + 1) * TUPLE_SIZE], 'big')
            posting_list.append((doc_id, tf))
        yield w, posting_list

In [4]:
base_path = "/content/indices/"
#gs://bucket_ir_engine/indices/Anchor_index.pkl
title_read = InvertedIndex.read_index(base_path, "Title_index")
body_read = InvertedIndex.read_index(base_path, "Body_index")
anchor_read = InvertedIndex.read_index(base_path, "Anchor_index")
print(title_read)
print(body_read)
print(anchor_read)

In [5]:
w_body, pl_body = zip(*posting_lists_iter(body_read, "/content/bins/Body_InvertedIndex/"))
w_title, pl_title = zip(*posting_lists_iter(title_read, "/content/bins/Title_InvertedIndex/"))
w_anchors, pl_anchors = zip(*posting_lists_iter(anchor_read, "/content/bins/Anchors_InvertedIndex/"))


[('/content/bins/Body_InvertedIndex/0_000.bin', 0)]
0


TypeError: ignored